In [1]:
!python --version

import os 

Python 3.10.4


In [4]:
from code.graph.layered_walk.utils import load_data

ModuleNotFoundError: No module named 'code.graph'; 'code' is not a package

In [2]:
import pickle 
import numpy as np 
import timeit 
import numba 

In [3]:
root = "/home/flavio/datasets/synthetic_layered_graph_1mil/"
layers = []
# nsample = 1000

In [4]:
# with open(root + "fake_" + "education" + "_adjacency_dict.pkl", "rb") as pkl_file:
#     edges = dict(pickle.load(pkl_file))
#     users = list(edges.keys())
#     users = users[:nsample]

In [5]:
layer_types = ["family", "colleague", "education", "neighbor", "household"]
layer_types = ["neighbor", "colleague"]
for ltype in layer_types:
    with open(root + "fake_" + ltype + "_adjacency_dict.pkl", "rb") as pkl_file:
        edges = dict(pickle.load(pkl_file))
        # edges_keep = dict((u, edges[u]) for u in users)
        layers.append(edges)

In [6]:
users = list(layers[0].keys())
len(users)

1000000

In [7]:
users[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [8]:
list(layers[0].keys())[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [10]:
# print(users[:4])
# for user in users: # XXX does this do anything? FIXME
#     user += 5
# print(users[:4])

[0, 1, 2, 3]
[0, 1, 2, 3]


In [11]:
# users = [u + 5 for u in users]
# print(users[:5])

[5, 6, 7, 8, 9]


In [13]:
node_layer_dict = {}
for user in users:
    node_layer_dict[user] = []
    
    for i, layer in enumerate(layers):
        if user in layer:
            if len(layer[user]) > 0:
                node_layer_dict[user].append(i)

In [14]:
a = list(node_layer_dict.keys())
print(a[0])
print(node_layer_dict[5])

5
[0, 1]


In [15]:
print(list(layers[0].keys())[:3])
print(layers[0][3])

[0, 1, 2]
[216591, 359203, 303294, 629913, 941856, 657939, 33155, 184951, 293721, 665878, 214931, 483588, 505018, 838030, 139399, 228960, 629783, 589567, 700786, 428120, 736350, 929797, 462671, 654239, 822538, 973284]


Numba does not work with dicts. Can we replace the following dicts?
- `layers`: a dict (layer type) of dicts (user id) of adjacency lists.
- node_layer_dict: a dict (user id) of layer ids, indicating whether the user has connections in this layer


could we not rewrite this as lists?
- `layers`: one layer is a list of adjacency lists, where the first adjacency list is from the first user
    - the user id refers to the order in the list
    - `layers` is a list of lists of lists 
- `node_layer_dict`: similar to a single layer: the first list indicates which layers are present for the first user

Chatgpt suggests to use numpy arrays for performance. Think; check this response.

In [16]:
# @numba.jit
def custom_sample(choice_set: list):
    "custom function to apply np.random.choice"
    if len(choice_set) == 0:
        return None 
    elif len(choice_set) == 1:
        chosen = choice_set[0]
    else:
        chosen = np.random.choice(choice_set)
    
    return chosen

In [17]:
# @numba.jit
def generate_walks(unique_users: list, walk_len: int, p: float = 0.8):
    "Generate one random walk for each user"
    num_users = len(unique_users)
    random_nums = np.random.rand(num_users, walk_len)

    rows = []
    for user_idx, user in enumerate(unique_users):   
        current_node = user
        
        layer_indices = node_layer_dict[current_node]
        layer_index = custom_sample(layer_indices)
        if not layer_index:
            break

        current_layer = layers[layer_index]
             
        walk = [user]
        while len(walk) < walk_len:
            layer_indices = node_layer_dict[current_node]
        
            roll = random_nums[user_idx][len(walk)]
            
            if roll > p:
                layer_index = custom_sample(layer_indices)
                if not layer:
                    break
                
            adjacent_nodes = current_layer[current_node]

            # Layer index should encode the layer type in an integer 0-4
            walk.append(layer_index)
            
            next_node = custom_sample(adjacent_nodes)
            if not next_node:
                break
            
            walk.append(next_node)
            current_node = next_node
        #assert len(walk) == walk_len, print(len(walk))
        rows.append(walk)

    return rows

    

In [18]:
walk_len = 40
num_walks = 5
a = generate_walks(unique_users=users, walk_len=walk_len)
# call this function num_walks times

In [19]:
%timeit generate_walks(unique_users=users, walk_len=walk_len)

768 ms ± 97.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
def create_many_walks(users: list, walk_len: int, num_walks: int):
    out = []
    for _ in range(num_walks):
        walks = generate_walks(unique_users=users, walk_len=walk_len)
        out.append(walks)
    return out 

In [28]:
%timeit create_many_walks(users=users, walk_len=walk_len, num_walks=num_walks)

799 ms ± 4.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Try with typed dict

In [133]:
@numba.njit
def custom_sample(choice_set: list):
    "custom function to apply np.random.choice"
    if len(choice_set) == 0:
        return -1 
    elif len(choice_set) == 1:
        chosen = choice_set[0]
    else:
        chosen = np.random.choice(choice_set)
    
    return np.int32(chosen)

In [ ]:
# cast layers, a list of dicts, into a list of numba typed dicts
# cast node_layer_dict, a dict, into a numba typed dict

In [121]:
from numba.typed import Dict, List
from numba.core import types


In [137]:

node_layer_dict_numba = Dict.empty(
    key_type=types.int32,
    value_type=types.int32[:]
)   
i = 0
for k, v in node_layer_dict.items():
    # if i > 10:
    #     break 
    k = types.int32(k)
    node_layer_dict_numba[k] = np.asarray(v, dtype=np.int32)
    i += 1


In [138]:
layers_numba = []
for layer in layers: 
    layer_numba = Dict.empty(
        key_type=types.int32,
        value_type=types.int32[:]
    )
    i = 0
    for k, v in layer.items():
        # if i > 10:
        #     break 
        k = types.int32(k)
        layer_numba[k] = np.asarray(v, dtype=np.int32)
        i += 1
        layers_numba.append(layer_numba)


In [149]:
@numba.njit
def single_walk(start_node: types.int32,
                walk_len: int, 
                node_layer_dict: numba.typed.Dict, 
                layers: list,
                p: float=0.8):
    """Create a single random walk starting at one node.
    
    Args:
        start_node: the node from which to start
        walk_len: the length of the random walk 
        node_layer_dict: dictionary indicating the layer indices in which each node as at least one edge.
        layers: list of numba.typed.Dict. Each layer is an edge list, indicating the connected nodes for each node. 
        p: probability of resampling the layer. 
    
    Returns:
        np.array: a sequence of node identifiers
    """
    current_node = start_node
    # walk = List.empty_list(types.int32)
    # walk.append(start_node)
    walk = [start_node]

    layer_indices = node_layer_dict[current_node]
    layer_index = custom_sample(layer_indices)
    if layer_index == -1:
        return walk

    # walk.append(start_node)
    for draw in np.random.rand(walk_len, 1):
    # while len(walk) < walk_len:
        draw = draw[0]
        layer_indices = node_layer_dict[current_node]
        # roll = random_nums.pop()

        if draw > p:
            layer_index = custom_sample(layer_indices)
            if layer_index == -1:
                break

        current_layer = layers[layer_index]
        adjacent_nodes = current_layer[current_node]

        walk.append(-layer_index - 1) # the first node is indicated by 0
        next_node = custom_sample(adjacent_nodes)
        if next_node == -1:
            break
        
        walk.append(next_node)
        current_node = next_node

    return walk 


In [122]:
# List.empty_list(types.int32)

ListType[int32]([, ...])

In [153]:
a = single_walk(
    start_node=10,
    walk_len=5,
    node_layer_dict=node_layer_dict_numba,
    layers=layers_numba
)


### ~~Now try with numpy arrays~~

We first store the adjacency information in two separate arrays:
- for each layer, a 1d array with the horizontally stacked adjacencies
- for each layer, a 1d array with the the start index of i's neighbors in the adjacencies
    - since arrays for each have the same shape (= number of nodes), we convert it into a 2d array of shape `[n_layers, n_users]`

I will shift the node ids when generating the walks. This keeps the code for the arrays simpler.

In [20]:
adjacencies = []
adj_idx = []
for current_layer in layers:
    cur_adjacencies = []
    # cur_adj_idx = [0] # let's implicitly assume the starting index for the first person is 0
    cur_adj_idx = []
    for i, (idx, neighbors) in enumerate(current_layer.items()):
        assert i == idx
        if len(cur_adj_idx) == 0:
            new_idx = len(neighbors)
        else:
            new_idx = cur_adj_idx[-1] + len(neighbors)
        cur_adj_idx.append(new_idx)
        cur_adjacencies += neighbors

    adj_idx.append(cur_adj_idx)
    adjacencies.append(np.array(cur_adjacencies))


adj_idx = np.array(adj_idx)


We also need to store the `node_layer_dict` in a 2d array. The rows indicate users, the columns indicate they layer id. If `True`, user `i` has neighbors in layer `j`.

In [21]:
nodes = list(layers[0].keys())
n_layers = len(layers)
nodes_and_layers = np.zeros((len(nodes), n_layers), dtype=np.bool_)
# nodes_and_layers = np.zeros((len(nodes), n_layers))

In [22]:
nodes_and_layers[0]

array([False, False])

In [23]:
for node in nodes:
    node_layer = []
    for i, adjacency in enumerate(adjacencies): 
        if adjacency[node] < adjacency[node+1]:
            node_layer.append(i)
    nodes_and_layers[node, node_layer] = 1
   


Now we can re-implement the generation of the walks using the three data structures
- `nodes_and_layers`, a array indicating which user has neighbors in which layer
- `adjacencies`, a list of arrays with adjacency information
- `adj_idx`, an array of starting positions for user `i` in `adjacencies`

In [24]:
node = 0
a = np.where(nodes_and_layers[node, :])
a
# a = np.squeeze(a)
# a.shape
# a[0].tolist()


(array([0]),)

In [30]:
@numba.jit
def custom_sample(choice_set: list):
    "custom function to apply np.random.choice"
    # print(choice_set)
    if len(choice_set) == 0:
        return -1 
    elif len(choice_set) == 1:
        chosen = choice_set[0]
    else:
        chosen = np.random.choice(choice_set)
    
    return chosen

In [35]:
# @numba.jit(nopython=True)
def gen_walks_np(adjacencies: list, 
                 adj_idx: np.array, 
                 nodes_and_layers: np.array, 
                 walk_len: int, 
                 p: float=0.8):
    n_nodes = nodes_and_layers.shape[0]
    walks = np.zeros((n_nodes, walk_len), dtype=np.int32)
    # walks = np.full((n_nodes, walk_len), -1, dtype=np.int32)
    random_nums = np.random.rand(n_nodes, walk_len)
    for node in range(n_nodes):
        walk = np.full(walk_len, -1, dtype=np.int32)
        available_layers = np.where(nodes_and_layers[node, :])[0]
        # print(f"available layers at start: {available_layers}")
        layer_index = custom_sample(available_layers)
        if layer_index == -1:
            walks[node, :] = walk
            continue 

        current_layer = adjacencies[layer_index]
        current_adj_idx = adj_idx[layer_index, :]
    

        current_node = node 
        # walk = walks[node, :]
        walk[0] = node + 5

        walk_pos = 1
        while walk_pos < walk_len-1: # XXX b/c we add *two* items in each loop, we need to stop earlier
            available_layers = np.where(nodes_and_layers[current_node, :])[0]
            roll = random_nums[node, walk_pos - 1]
            if roll > p:
                layer_index = custom_sample(available_layers)
                if layer_index == -1: 
                    break
            
            if node == 0:
                end_idx = current_adj_idx[0]
                adjacent_nodes = current_layer[:end_idx]
            else:
                start_idx = current_adj_idx[node -1]
                end_idx = current_adj_idx[node]
                adjacent_nodes = current_layer[start_idx:end_idx]


            next_node = custom_sample(adjacent_nodes)

            if next_node == -1:
                break

            walk[walk_pos] = layer_index
            walk[walk_pos + 1] = next_node + 5
            current_node = next_node
            walk_pos += 2
        
        walks[node] = walk
    return walks


In [36]:

a = gen_walks_np(adjacencies=adjacencies,
                 adj_idx=adj_idx,
                 nodes_and_layers=nodes_and_layers,
                 walk_len=3)

In [37]:
walk_len = 40
num_walks = 5

In [38]:
%timeit gen_walks_np(adjacencies=adjacencies, adj_idx=adj_idx, nodes_and_layers=nodes_and_layers, walk_len=walk_len)

KeyboardInterrupt: 

In [60]:

@numba.jit
def test_numba(nodes_and_layers):
    n_nodes = nodes_and_layers.shape[0]
    for node in range(n_nodes):
        available_layers = np.where(nodes_and_layers[node, :])[0]
        layer_index = custom_sample(available_layers)
    

In [62]:
test_numba(nodes_and_layers)

In [ ]:

# @numba.jit
def generate_walks(unique_users: list, walk_len: int, p: float = 0.8):
    "Generate one random walk for each user"
    num_users = len(unique_users)
    random_nums = np.random.rand(num_users, walk_len)

    rows = []
    for user_idx, user in enumerate(unique_users):   
        current_node = user
        
        layer_indices = node_layer_dict[current_node]
        layer_index = custom_sample(layer_indices)
        if not layer_index:
            break

        current_layer = layers[layer_index]
             
        walk = [user]
        while len(walk) < walk_len:
            layer_indices = node_layer_dict[current_node]
        
            roll = random_nums[user_idx][len(walk)]
            
            if roll > p:
                layer_index = custom_sample(layer_indices)
                if not layer_index:
                    break
                
            adjacent_nodes = current_layer[current_node]

            # Layer index should encode the layer type in an integer 0-4
            walk.append(layer_index)
            
            next_node = custom_sample(adjacent_nodes)
            if not next_node:
                break
            
            walk.append(next_node)
            current_node = next_node
        #assert len(walk) == walk_len, print(len(walk))
        rows.append(walk)

    return rows

    